In [1]:
from _shared import *

### Задача  1. Оценка разных статистик

В лекции мы выяснили, что в эксперименте «Refactoring backend» в экспериментальной группе среднее время загрузки увеличилось, а 99% квантиль уменьшился. Проверьте значимость отличий других квантилей.

Данные эксперимента «Refactoring backend»:

`2022-04-13/2022-04-13T12_df_web_logs.csv` и `2022-04-13/experiment_users.csv`.

Эксперимент проводился с __2022-04-05__ по __2022-04-12__.

Измерения времени обработки запросов считаем независимыми. 
При проверке используйте нормальный доверительный интервал.

Выберете верные утверждения:
- Квантиль 0.70 статистически значимо изменился
- Квантиль 0.74 статистически значимо изменился
- Квантиль 0.78 статистически значимо изменился
- Квантиль 0.82 статистически значимо изменился
- Квантиль 0.86 статистически значимо изменился
- Квантиль 0.90 статистически значимо изменился
- Квантиль 0.95 статистически значимо изменился
- Квантиль 0.99 статистически значимо изменился
- Квантиль 0.999 статистически значимо изменился
- Квантиль 0.9999 статистически значимо изменился

In [2]:
# df_web_logs = read_from_database('2022-04-13/2022-04-13T12_df_web_logs.csv') #Fails due to the bad naming.
df_web_logs = get_data_subset(
    df=pd.read_csv('./data/2022-04-13T12_df_web_logs.csv', parse_dates=[2]),
    begin_date='2022-04-05',
    end_date='2022-04-12'
)

df_web_logs.head()

,user_id,page,date,load_time
2111340,190c2f,m,2022-04-05 00:00:38,66.2
2111341,e65269,m,2022-04-05 00:01:25,60.0
2111342,f0d8b6,b,2022-04-05 00:01:41,72.3
2111343,ea0e98,b,2022-04-05 00:05:44,69.9
2111344,ea0e98,m,2022-04-05 00:06:50,53.1


In [3]:
# experiment_users = read_from_database('experiment_users.csv') #Different.
experiment_users = pd.read_csv('./data/experiment_users.csv')

experiment_users.head()

,user_id,pilot
0,c36b2e,0
1,20336e,0
2,034652,0
3,e98e3b,0
4,3f1105,0


In [4]:
df_sub = (
    df_web_logs
    .merge(
        right=experiment_users,
        how='inner',
        on='user_id'
    ).loc[:, ['pilot', 'load_time']]
)

df_sub.head()

,pilot,load_time
0,1,60.0
1,1,71.7
2,1,66.1
3,1,73.6
4,1,68.7


In [5]:
df_sub.groupby('pilot').describe()

load_time                                                      
          count       mean         std  min   25%   50%   75%     max
pilot                                                                
0       19270.0  74.564536  119.863299  0.0  61.8  70.1  78.1  4102.8
1       19666.0  76.937044  112.503063  0.0  67.7  73.0  78.6  4176.6

In [6]:
def get_quantile_diff(values_a, values_b, quantile: float=0.999):
    return np.quantile(values_b, quantile) - np.quantile(values_a, quantile)


def get_bootstrap_point_estimates(values_a, values_b, quantile: float=0.999, N: int=1_000):
    bootstrap_point_estimates = []
    
    for _ in range(N):
        bootstrap_values_a = np.random.choice(values_a, size=len(values_a), replace=True)
        bootstrap_values_b = np.random.choice(values_b, size=len(values_b), replace=True)
        bootstrap_point_estimates.append(get_quantile_diff(bootstrap_values_a, bootstrap_values_b, quantile))
            
    return bootstrap_point_estimates


# Optimized code.
def get_bootstrap_confidence_interval(
    bootstrap_values: np.array,
    point_estimate: float=0.0,
    ci_type: str='normal',
    alpha: float=0.05
):
    """
    Bulding confidence interval.
    
    :param bootstrap_values: bootstrapped values of the metric.
    :param point_estimate: point estimate of the metric.
    :param ci_type: confidence interval type.
    :param alpha: level of significance.
    
    :returns: (left, right) - tuple containing confidence interval ends.
    """
    left, right = None, None
    
    if ci_type == 'normal':
        ppf_ = stats.norm.ppf(1.0 - alpha / 2)
        std_ = np.std(bootstrap_values)
        
        left, right = point_estimate - ppf_ * std_, point_estimate + ppf_ * std_

    elif ci_type == 'percentile':
        left, right = np.quantile(bootstrap_values, [alpha / 2, 1.0 - alpha / 2])
    
    elif ci_type == 'pivotal':
        left, right = 2 * point_estimate - np.quantile(bootstrap_values, [1.0 - alpha / 2, alpha / 2])
    
    return (left, right)


# def get_ci_bootstrap_normal(boot_metrics: np.array, pe_metric: float, alpha: float=0.05):
#     """Строит нормальный доверительный интервал.

#     boot_metrics - значения метрики, полученные с помощью бутстрепа
#     pe_metric - точечная оценка метрики
#     alpha - уровень значимости
    
#     return: (left, right) - границы доверительного интервала.
#     """
#     c = stats.norm.ppf(1 - alpha / 2)
#     se = np.std(boot_metrics)
#     left, right = pe_metric - c * se, pe_metric + c * se
#     return left, right


# def get_ci_bootstrap_percentile(boot_metrics: np.array, pe_metric: float, alpha: float=0.05):
#     """Строит доверительный интервал на процентилях.

#     boot_metrics - значения метрики, полученные с помощью бутстрепа
#     pe_metric - точечная оценка метрики
#     alpha - уровень значимости
    
#     return: (left, right) - границы доверительного интервала.
#     """
#     left, right = np.quantile(boot_metrics, [alpha / 2, 1 - alpha / 2])
#     return left, right


# def get_ci_bootstrap_pivotal(boot_metrics: np.array, pe_metric: float, alpha: float=0.05):
#     """Строит центральный доверительный интервал.

#     boot_metrics - значения метрики, полученные с помощью бутстрепа
#     pe_metric - точечная оценка метрики
#     alpha - уровень значимости
    
#     return: (left, right) - границы доверительного интервала.
#     """
#     right, left = 2 * pe_metric - np.quantile(boot_metrics, [alpha / 2, 1 - alpha / 2])
#     return left, right

In [7]:
quantile_list = (
    0.70,
    0.74,
    0.78,
    0.82,
    0.86,
    0.90,
    0.95,
    0.99,
    0.999,
    0.9999
)

ci_types = ('normal', )#'percentile', 'pivotal')

quantile_ci = {}

for quantile_ in quantile_list:
    sample_a = df_sub.loc[df_sub['pilot'] == 0, 'load_time'] 
    sample_b = df_sub.loc[df_sub['pilot'] == 1, 'load_time']
    
    bpe = get_bootstrap_point_estimates(
        values_a=sample_a,
        values_b=sample_b,
        quantile=quantile_,
        N=10_000
    )
    
    for ci_type_ in ci_types:
        quantile_ci[(quantile_, ci_type_)] = get_bootstrap_confidence_interval(
            bootstrap_values=bpe,
            point_estimate=get_quantile_diff(sample_a, sample_b, quantile_),
            ci_type=ci_type_,
            alpha=0.05
        )


In [8]:
for (k, v) in quantile_ci.items():
    includes_0 = v[0] < 0 and v[1] > 0
    
    if includes_0:
        checked_char = '_'
    else:
        checked_char = 'X'
    
    print("""[{}] {:.4f}, {}: ({:.2f}, {:.2f}) - Includes 0.0? [{}] / Significant change? [{}]
    """.format(checked_char, k[0], k[1], v[0], v[1], includes_0, not includes_0)
    )

[X] 0.7000, normal: (0.72, 1.28) - Includes 0.0? [False] / Significant change? [True]
    
[X] 0.7400, normal: (0.31, 0.89) - Includes 0.0? [False] / Significant change? [True]
    
[_] 0.7800, normal: (-0.30, 0.30) - Includes 0.0? [True] / Significant change? [False]
    
[X] 0.8200, normal: (-1.01, -0.39) - Includes 0.0? [False] / Significant change? [True]
    
[X] 0.8600, normal: (-1.56, -0.84) - Includes 0.0? [False] / Significant change? [True]
    
[X] 0.9000, normal: (-2.48, -1.72) - Includes 0.0? [False] / Significant change? [True]
    
[X] 0.9500, normal: (-3.89, -3.02) - Includes 0.0? [False] / Significant change? [True]
    
[X] 0.9900, normal: (-7.30, -5.30) - Includes 0.0? [False] / Significant change? [True]
    
[_] 0.9990, normal: (-1979.33, 1201.06) - Includes 0.0? [True] / Significant change? [False]
    
[_] 0.9999, normal: (-417.88, 481.30) - Includes 0.0? [True] / Significant change? [False]
    


In [9]:
# Solution
import os
from datetime import datetime
import numpy as np
import pandas as pd
from scipy import stats

URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-13/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

df_web_logs = read_database('2022-04-13T12_df_web_logs.csv')
df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])
df_users = read_database('experiment_users.csv')

begin_date = datetime(2022, 4, 5)
end_date = datetime(2022, 4, 12)
df_metrics = (
    df_web_logs
    [(df_web_logs['date'] >= begin_date) & (df_web_logs['date'] < end_date)]
    [['user_id', 'load_time']]
)
df = pd.merge(df_users, df_metrics, on='user_id', how='left')[['pilot', 'load_time']]

def get_ci_bootstrap_normal(boot_metrics: np.array, pe_metric: float, alpha: float=0.05):
    """Строит нормальный доверительный интервал.

    boot_metrics - значения метрики, полученные с помощью бутстрепа
    pe_metric - точечная оценка метрики
    alpha - уровень значимости
    
    return: (left, right) - границы доверительного интервала.
    """
    c = stats.norm.ppf(1 - alpha / 2)
    se = np.std(boot_metrics)
    left, right = pe_metric - c * se, pe_metric + c * se
    return left, right

B = 1000
alpha = 0.05
values_a = df[df['pilot'] == 0]['load_time'].values
values_b = df[df['pilot'] == 1]['load_time'].values

quantiles = np.array([0.7, 0.74, 0.78, 0.82, 0.86, 0.9, 0.95, 0.99, 0.999, 0.9999])
for q in quantiles:
    pe = np.quantile(values_b, q) - np.quantile(values_a, q)
    bootstrap_values_a = np.random.choice(values_a, (B, len(values_a)), True)
    bootstrap_metrics_a = np.quantile(bootstrap_values_a, q, axis=1)
    bootstrap_values_b = np.random.choice(values_b, (B, len(values_b)), True)
    bootstrap_metrics_b = np.quantile(bootstrap_values_b, q, axis=1)
    bootstrap_stats = bootstrap_metrics_b - bootstrap_metrics_a
    left, right = get_ci_bootstrap_normal(bootstrap_stats, pe, alpha)
    print(f'q = {q}, ci = [{left:0.3f}, {right:0.3f}], is effect = {not(left < 0 < right)} ')


q = 0.7, ci = [0.726, 1.274], is effect = True 
q = 0.74, ci = [0.296, 0.904], is effect = True 
q = 0.78, ci = [-0.301, 0.301], is effect = False 
q = 0.82, ci = [-1.014, -0.386], is effect = True 
q = 0.86, ci = [-1.542, -0.858], is effect = True 
q = 0.9, ci = [-2.472, -1.728], is effect = True 
q = 0.95, ci = [-3.890, -3.020], is effect = True 
q = 0.99, ci = [-7.289, -5.311], is effect = True 
q = 0.999, ci = [-1982.275, 1204.005], is effect = False 
q = 0.9999, ci = [-447.908, 511.333], is effect = False 


### Задача 2. Функция оценки эксперимента с помощью bootstrap

In [10]:
"""
Реализуйте функцию run_bootstrap.

Шаблон решения
"""

import numpy as np
from scipy import stats


def run_bootstrap(bootstrap_metrics, pe_metric, alpha, bootstrap_ci_type):
    """Строит доверительный интервал и проверяет значимость отличий с помощью бутстрепа.
    
    :param bootstrap_metrics (np.array): множество значений статистики теста,
        посчитанные на бутстрепных выборках.
    :param pe_metric (float): значение статистики теста посчитанное по исходным данным.
    :param alpha (float): уровень значимости.
    :param bootstrap_ci_type (str): способ построения доверительного интервала.
        Возможные значения ['normal', 'percentile', 'pivotal'].
    :return ci, pvalue:
        ci [float, float] - границы доверительного интервала.
        pvalue (float) - 0 если есть статистически значимые отличия, иначе 1. Настоящее
        pvalue для произвольного способа построения доверительного интервала с помощью
        бутстрепа вычислить не тривиально. Будем использовать краевые значения 0 и 1.
    """
    left, right = None, None
    
    if bootstrap_ci_type == 'normal':
        ppf_ = stats.norm.ppf(1.0 - alpha / 2)
        std_ = np.std(bootstrap_metrics)
        
        left, right = pe_metric - ppf_ * std_, pe_metric + ppf_ * std_

    elif bootstrap_ci_type == 'percentile':
        left, right = np.quantile(bootstrap_metrics, [alpha / 2, 1.0 - alpha / 2])
    
    elif bootstrap_ci_type == 'pivotal':
        left, right = 2 * pe_metric - np.quantile(bootstrap_metrics, [1.0 - alpha / 2, alpha / 2])
    
    return ([left, right], float(left < 0 < right))


def generate_bootstrap_metrics(data_one, data_two, bootstrap_iter, bootstrap_agg_func):
    """Генерирует значения метрики, полученные с помощью бутстрепа.

    :param data_one, data_two (np.array): значения метрик в группах.
    :param design (Design): объект с данными, описывающий параметры эксперимента
    :param bootstrap_iter (int): количество итераций бутстрепа.
    :param bootstrap_agg_func (str): метрика эксперимента.
        Возможные значения ['mean', 'quantile 95'].
    :return bootstrap_metrics, pe_metric:
        bootstrap_metrics (np.array) - множество значений статистики теста,
            посчитанные на бутстрепных выборках.
        pe_metric (float) - значение статистики теста посчитанное по исходным данным.
    """
    bootstrap_data_one = np.random.choice(data_one, (len(data_one), bootstrap_iter))
    bootstrap_data_two = np.random.choice(data_two, (len(data_two), bootstrap_iter))
    if bootstrap_agg_func == 'mean':
        bootstrap_metrics = (
            bootstrap_data_two.mean(axis=0) - bootstrap_data_one.mean(axis=0)
        )
        pe_metric = data_two.mean() - data_one.mean()
        return bootstrap_metrics, pe_metric
    elif bootstrap_agg_func == 'quantile 95':
        q = 0.95
        bootstrap_metrics = (
            np.quantile(bootstrap_data_two, q, axis=0)
            - np.quantile(bootstrap_data_one, q, axis=0)
        )
        pe_metric = np.quantile(data_two, q) - np.quantile(data_one, q)
        return bootstrap_metrics, pe_metric
    else:
        raise ValueError('Неверное значение bootstrap_agg_func')
        

data_one, data_two = np.array([1, 3]), np.array([5, 7])
bootstrap_iter = 10
bootstrap_agg_func = 'mean'
bootstrap_metrics, pe_metric = generate_bootstrap_metrics(
    data_one, data_two, bootstrap_iter, bootstrap_agg_func
)
# bootstrap_metrics = np.array([6., 5., 3., 4., 5., 2., 6., 4., 4., 4.])
# pe_metric = 4.0
print_kwargs(
    bootstrap_metrics=np.array(bootstrap_metrics).round(),
    pe_metric=pe_metric
)

bootstrap_metrics = [4. 4. 4. 3. 4. 4. 4. 2. 4. 5.]
pe_metric = 4.0


In [11]:
bootstrap_metrics = np.arange(-90, 910)
pe_metric = 600.
alpha = 0.05
bootstrap_ci_types = ['normal', 'percentile', 'pivotal']
for bootstrap_ci_type in bootstrap_ci_types:
    ci, pvalue = run_bootstrap(bootstrap_metrics, pe_metric, alpha, bootstrap_ci_type)
    print(bootstrap_ci_type)
    print(f'ci = {np.array(ci).round()}, pvalue = {pvalue}')
# >>> normal: ci = [  34. 1166.], pvalue = 0.0
# >>> percentile: ci = [-65. 884.], pvalue = 1.0
# >>> pivotal: ci = [ 316. 1265.], pvalue = 0.0

normal
ci = [  34. 1166.], pvalue = 0.0
percentile
ci = [-65. 884.], pvalue = 1.0
pivotal
ci = [ 316. 1265.], pvalue = 0.0


In [12]:
# Solution
import numpy as np
from scipy import stats


def run_bootstrap(bootstrap_metrics, pe_metric, alpha, bootstrap_ci_type):
    if bootstrap_ci_type == 'normal':
        c = stats.norm.ppf(1 - alpha / 2) 
        se = np.std(bootstrap_metrics)
        left, right = pe_metric - c * se, pe_metric + c * se
    elif bootstrap_ci_type == 'percentile':
        left, right = np.quantile(bootstrap_metrics, [alpha / 2, 1 - alpha / 2])
    elif bootstrap_ci_type == 'pivotal':
        q = np.quantile(bootstrap_metrics, [alpha / 2, 1 - alpha / 2])
        right, left = 2 * pe_metric - q
    else:
        raise ValueError('Неверное значение bootstrap_ci_type')
    pvalue = float(left < 0 < right)
    return (left, right,), pvalue

In [13]:
x = 0

-1 < x < 1

True